In [14]:
import sys
sys.path.append('..')

import os
from sqlalchemy import create_engine, text
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
from shapely import wkb
from geopandas.tools import sjoin

from utilities_amigocloud import AmigocloudFunctions

In [15]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
PATH_CAT = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\SHP\CATASTRO_S13.shp'
PATH_XLSX_GRUPOS = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPO_COSECHA.xlsx'

In [16]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_planificacion():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_pulverizacion.planificacion_pulv
            WHERE procesado=false;
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def marcar_como_procesado(id_os):
    engine = obtener_engine()  # tu función que crea el engine
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.planificacion_pulv
                SET procesado = true
                WHERE id = :id_os
            """)
            conn.execute(query, {"id_os": id_os})
            print(f"✔️ id {id_os} marcado como procesado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")
    return None

def obtener_parte_diario_por_os(ors):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_pulverizacion.parte_diario_pulv where os = {ors}
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def convertir_a_multipolygon(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    return geometry

def convertir_a_wkb(polygon):
    wkb_data = wkb.dumps(polygon, hex=True)
    return wkb_data

def cargar_a_amigocloud(gdf):
    # repreyectar a WGS84
    gdf_pla_gral = gdf.to_crs(epsg=4326)
    # convertir poligonos a multipoligonos
    gdf_pla_gral['geom'] = gdf_pla_gral['geom'].apply(convertir_a_multipolygon)
    
    gdf_pla_gral['unidad_01'] = gdf_pla_gral['unidad_01'].astype(int)
    gdf_pla_gral['unidad_03'] = gdf_pla_gral['unidad_03'].astype(int)
    gdf_pla_gral['os'] = gdf_pla_gral['os'].astype(int)
    gdf_pla_gral['soca'] = gdf_pla_gral['soca'].astype(int)
    gdf_pla_gral['id'] = gdf_pla_gral['id'].astype(int)
    gdf_pla_gral['inst'] = gdf_pla_gral['inst'].astype(int)
    
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 35248
    for index, row in gdf_pla_gral.iterrows():
        wkb_hex = convertir_a_wkb(row['geom'])
        insert_sql = f"""
            INSERT INTO dataset_360912 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry)
            VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['os']}', ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
        """
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        print(f'Lote registrado en AmigoCloud')
    return None

In [17]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

In [18]:
gdf_cat = gpd.read_file(PATH_CAT)
gdf_plan = obtener_planificacion()
gdf_plan

,id,geom,codigo_canero,nombre_canero,obs,procesado
0,45,"MULTIPOINT (502517.487 8097072.990, 502757.166...",41594,AGROPECUARIA CAMPO DULCE S.R.L.,None,False


# CARGAR PARTE DIARIO

In [19]:
gdf_intersect_result = gpd.sjoin(gdf_cat, gdf_plan, how='inner')
os_para_cargar = list(set(gdf_intersect_result['id_right']))
os_para_cargar

[45]

In [20]:
gdf_result = gdf_intersect_result[['geometry', 'unidad_01', 'unidad_02', 'codigo_canero', 'nombre_canero', 'unidad_05', 'area', 'id_right', 'soca', 'zona']].copy()
gdf_result['area'] = gdf_result.geometry.area / 10000
gdf_result.rename(columns={
    'geometry': 'geom',
    'codigo_canero': 'unidad_03',
    'nombre_canero': 'unidad_04',
    'id_right': 'os',
    'zona': 'inst'
}, inplace=True)

gdf_result = gdf_result.set_geometry("geom")
gdf_result['unidad_01'] = gdf_result['unidad_01'].astype(int)
gdf_result['unidad_03'] = gdf_result['unidad_03'].astype(int)
gdf_result['os'] = gdf_result['os'].astype(int)
gdf_result['soca'] = gdf_result['soca'].astype(int)
gdf_result['inst'] = gdf_result['inst'].astype(int)

In [21]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst
5858,"POLYGON ((502460.228 8097214.480, 502475.564 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L6,0.854007,45,6,50
5872,"POLYGON ((502892.921 8096050.883, 502876.956 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L4.2,0.572246,45,6,50
5873,"POLYGON ((502872.920 8096009.835, 502667.935 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L2,22.500444,45,6,50
7359,"POLYGON ((502760.998 8095561.239, 502741.869 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L18.1,2.090157,45,5,50
8422,"POLYGON ((503334.416 8096631.203, 503334.416 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L9.1,27.161171,45,6,50
9057,"POLYGON ((502638.769 8096661.307, 502449.857 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L3,21.372012,45,6,50
9058,"POLYGON ((502697.358 8096140.943, 502692.847 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L4.1,12.443581,45,6,50
9061,"POLYGON ((502703.026 8097043.351, 502767.251 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L7,2.723805,45,6,50
9064,"POLYGON ((502791.591 8096633.345, 502998.362 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L8,17.419571,45,6,50
9066,"POLYGON ((502251.287 8096954.184, 502399.709 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L16,15.902201,45,7,50


In [22]:
# GARANTIZAR LA INST DEL CAÑERO
# cargar grupos de cosecha
df_grupos = pd.read_excel(PATH_XLSX_GRUPOS, sheet_name='CODIGOS')
df_grupos = df_grupos[df_grupos['INSTITUCION'].notna()]
df_grupos['CODIGO CAÑERO'] = df_grupos['CODIGO CAÑERO'].astype(int)
df_grupos['INSTITUCION'] = df_grupos['INSTITUCION'].astype(int)

# Crear un diccionario de mapeo: {codigo_cañero: institucion}
mapa_institucion = df_grupos.set_index('CODIGO CAÑERO')['INSTITUCION'].to_dict()
# Reemplazar los valores de 'inst' en gdf_result usando el diccionario
# a partir del diccionario, busca el codigo cañero (unidad_03), y reempalza la institucion
gdf_result['inst'] = gdf_result['unidad_03'].map(mapa_institucion)

In [23]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst
5858,"POLYGON ((502460.228 8097214.480, 502475.564 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L6,0.854007,45,6,50
5872,"POLYGON ((502892.921 8096050.883, 502876.956 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L4.2,0.572246,45,6,50
5873,"POLYGON ((502872.920 8096009.835, 502667.935 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L2,22.500444,45,6,50
7359,"POLYGON ((502760.998 8095561.239, 502741.869 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L18.1,2.090157,45,5,50
8422,"POLYGON ((503334.416 8096631.203, 503334.416 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L9.1,27.161171,45,6,50
9057,"POLYGON ((502638.769 8096661.307, 502449.857 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L3,21.372012,45,6,50
9058,"POLYGON ((502697.358 8096140.943, 502692.847 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L4.1,12.443581,45,6,50
9061,"POLYGON ((502703.026 8097043.351, 502767.251 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L7,2.723805,45,6,50
9064,"POLYGON ((502791.591 8096633.345, 502998.362 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L8,17.419571,45,6,50
9066,"POLYGON ((502251.287 8096954.184, 502399.709 8...",259,SANTA ANA DE PAILON,41594,AGROPECUARIA CAMPO DULCE S.R.L.,L16,15.902201,45,7,50


In [24]:
# agregar nuevos registros a la base de datos
gdf_result.to_postgis("parte_diario_pulv", obtener_engine(), schema="drones_pulverizacion", if_exists="append")

In [25]:
os_para_cargar

[45]

In [26]:
# CARGAR A AMIGOCLOD
for i in os_para_cargar:
    plan_os = obtener_parte_diario_por_os(i)
    cargar_a_amigocloud(plan_os)
    marcar_como_procesado(i)

Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
✔️ id 45 marcado como procesado.
